In [46]:
'''
Code intended to evaluate the performances of Orbslam3 and DSO using custom dataset in TUM format.
Requirements.
1) Container containing DSO (custom)
2) Container containing ORBSLAM3 (custom)
3) Folder containing the datasets
4) Evo tools ;D
'''
import docker
from docker.errors import APIError
import os
import sys
import tarfile
import numpy as np
import pandas as pd
import yaml
import logging
from tqdm import tqdm
from datetime import datetime
import shutil

In [47]:
def copy_to_container(container, container_dest_path, dataset_dir):
    print("Copying '{}' in \"{}\" ".format(dataset_dir,container_dest_path))
    temp_dir_processing = os.path.join(os.getcwd(), "temp","".join(["curr_dataset",".tar.gz"]))

    print("\t - 1) Compressing {}".format(temp_dir_processing))
    with tarfile.open(temp_dir_processing, "w:gz") as tar:
        tar.add(dataset_dir, arcname=os.path.basename(dataset_dir))
    print("\t\tDone!")

    print("\t - 2) Copying into DSO containter: {}".format(container.name))
    with open(temp_dir_processing, 'rb') as fd:
        ok = dso_container.put_archive(path=container_dest_path, data=fd)
        if not ok:
            raise Exception('Put file failed')
        else:
            print("\t\tDone!")
    if os.path.exists(temp_dir_processing):
        print("\t - 3) Removing the temp file:{}".format(temp_dir_processing))
        os.remove(temp_dir_processing)
        print("\t\tDone!")
    else:
        print('Unable to remove {} \n this not a critucal problem, proceed...'.format(temp_dir_processing))

In [48]:
######################################################################
# LOAD CONFIG
######################################################################
sess_root_path = os.getcwd()
config_path = os.path.join(sess_root_path, "config")
docker_config = os.path.join(config_path, "docker_containers.yaml")
local_config = os.path.join(config_path, "local_host_params.yaml")
print("Current config path: {}".format(config_path))
try:
    with open(docker_config, "r") as yamlfile:
        docker_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t-Docker config loaded;")
except OSError:
    print("Could not open/read file:", docker_config)
    sys.exit()

try:
    with open(local_config, "r") as yamlfile:
        local_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t-Local config loaded;")
except OSError:
    print("Could not open/read file:", local_config)
    sys.exit()

# Get the configurations for the containers
dso_container_config = docker_data_cfg["Containers"][0]["DSO"]
orb_container_config = docker_data_cfg["Containers"][1]["ORBSLAM3"]

print("DSO CONTAINTER CONFIGURATIONS")
print("\t{}".format(dso_container_config))

print("ORBSLAM CONTAINTER CONFIGURATIONS")
print("\t{}".format(orb_container_config))


Current config path: /home/croc/Documenti/Repositories/evoTools/notebooks/config
	-Docker config loaded;
	-Local config loaded;
DSO CONTAINTER CONFIGURATIONS
	{'Id': '7a0bedaae163', 'Names': 'test_dso', 'datasetFolder': '/root/Archive/Dataset/', 'Algorithm': {'binPath': '/root/Programs/dso/build/bin/', 'exec': 'dso_dataset_params', 'output_file': 'result.txt', 'configs': {'files': 'images.zip', 'calib': 'camera.txt', 'gamma': 'pcalib.txt', 'vignette': 'vignette.png'}, 'params': {'ImmatureDensity': 400, 'PointDensity': 800, 'minFrames': 5, 'maxFrames': 7, 'maxOptIterations': 6, 'minOptIterations': 1, 'speed': 0, 'mode': 0, 'nogui': 1, 'quiet': 1}}}
ORBSLAM CONTAINTER CONFIGURATIONS
	{'Id': '4ecad379b081', 'Names': 'test_orbslam3', 'datasetFolder': '/root/Archive/Dataset/', 'Algorithm': {'binPath': '/root/Programs/ORB_SLAM3/Examples/Monocular/'}}


In [49]:
######################################################################
# CREATE SESSION FOLDER
######################################################################
# datetime object containing current date and time

now = datetime.now()
sess_string = ''.join(["session_",now.strftime("%d_%m_%Y_%H_%M_%S")])
session_folder = os.path.join(sess_root_path, sess_string)
print("Creating Session folder: {}".format(session_folder))
try:
    os.mkdir(session_folder)
    print("\tDone!")
except OSError as error:
    print("Fatal Error: {}".format(error))
    sys.exit(1)

Creating Session folder: /home/croc/Documenti/Repositories/evoTools/notebooks/session_29_09_2023_15_43_34
	Done!


In [50]:
######################################################################
# DOCKER SET-UP
######################################################################

# Get the client
client_dk = docker.from_env()

# Retrieve the containers
orb_container = [cont for cont in client_dk.containers.list(all=True) if cont.short_id == orb_container_config["Id"]][0]
dso_container = [cont for cont in client_dk.containers.list(all=True) if cont.short_id == dso_container_config["Id"]][0]


In [51]:
# Start the two containers
if orb_container.status == 'exited':
    orb_container.start()
    print("Container {} Started".format(orb_container.name))
elif orb_container.status =="running":
     print("Container {} Already started".format(orb_container.name))

if dso_container.status == 'exited':
    dso_container.start()
    print("Container {} Started".format(dso_container.name))
elif dso_container.status =="running":
    print("Container {} Already started".format(dso_container.name))

Container test_orbslam3 Already started
Container test_dso Already started


In [52]:
######################################################################
# DATASET LOADING
######################################################################

#Common Parameters
datasets_root_path = local_data_cfg["Datasets"]["root_path"]
datasets_map_folder = local_data_cfg["Datasets"]["map_folder"]
datasets_data_folder = local_data_cfg["Datasets"]["data_folder"]
datasets_eva_folder = local_data_cfg["Datasets"]["out_eva_folder"]
dataset_gt_suffix = local_data_cfg["Datasets"]["GtFileSuffix"].split("*")[-1]
#Datasets selected
dataset_list = local_data_cfg["Datasets"]["Scenarios"]

In [53]:
# Sub-Datasets Discovering
dataset_selected = list(dataset_list.keys())[0]
folder_dataset_sel=dataset_list[dataset_selected]["folder_name"]
path_dataset = os.path.join(datasets_root_path, folder_dataset_sel)
if os.path.isdir(path_dataset):
    elem = [elem for elem in os.listdir(os.path.join(path_dataset, datasets_data_folder))]
    print("Discovering subsets of {}: {}".format(dataset_selected,elem))
else:
    elem = []

Discovering subsets of Corridor_A: ['Corridor_A_D_190', 'Corridor_A_D_60', 'Corridor_A_D_255', 'Corridor_A_D_85', 'Corridor_A_D_127', 'Corridor_A_L', 'Corridor_A_D_25']


In [54]:


# TODO: This must be extended to ORBSLAM and for all the subsets
# We'll try to process just the CorridorA
curr_dataset = elem[5]
algorithm = "DSO"
print("\n"+ "*"*50 + "\n Processing with {} the {} dataset".format(algorithm, curr_dataset) +"\n"+ "*"*50+"\n")

# Compose the full path of the desired Dataset
path_sub_dataset = os.path.join(path_dataset, datasets_data_folder, curr_dataset)

# Create the dataset folder results and copy the gt file
dataset_out_folder= os.path.join(session_folder, curr_dataset)
try:
    print("Creating Dataset folder: {}".format(dataset_out_folder))
    os.mkdir(dataset_out_folder)
    print("\tDone!")
    gt_filename = "".join([curr_dataset, dataset_gt_suffix])
    file_gt = os.path.join(path_sub_dataset,gt_filename)
    print("Copying the gt file {} ----> {}".format(file_gt,dataset_out_folder))
    shutil.copy(file_gt, dataset_out_folder)
    print("\tDone!")
except OSError as error:
    print("Fatal Error: {}".format(error))
    sys.exit(1)

# Get the destination directory
destination_path = dso_container_config["datasetFolder"]

##### END COMMON CODE, NOW IT WILL REFER JUST TO DSO#############
# Transfer the dataset into the container
copy_to_container(dso_container, destination_path, path_sub_dataset)
container_actual_data = os.path.join(destination_path, curr_dataset)


# Preparing the cmd for DSO:
dso_container_config_algorithm = dso_container_config["Algorithm"]
dso_bin_path = dso_container_config_algorithm["binPath"]

# 1) Change directory command
dso_ch_dir_string="".join(["cd", " ", dso_bin_path])

# 2a) Executable command
executable_string = str(os.path.join(".",  dso_container_config_algorithm["exec"]))

# 2b) Configs command
config_string = ""
first = True
for config in dso_container_config_algorithm["configs"]:
    config_value=dso_container_config_algorithm["configs"][config]
    if first:
        config_string+="".join([str(config),"=",str(os.path.join(container_actual_data,config_value))])
        first = False
    else:
        config_string+="".join([" ",str(config),"=",str(os.path.join(container_actual_data,config_value))])

# 2c) Parameters command
param_string = ""
first = True
for param in dso_container_config_algorithm["params"]:
    param_value=dso_container_config_algorithm["params"][param]
    if first:
        param_string+="".join([str(param),"=",str(param_value)])
        first = False
    else:
        param_string+="".join([" ",str(param),"=",str(param_value)])

# 2d) Command String
dso_command_string = "".join([executable_string, " ", config_string, " ", param_string])


# 3)  command string
command_string = "".join(["bash -c \"", dso_ch_dir_string, " && ",  dso_command_string, "\""])
print("\nOn container \"{}\" execute the following command:  \n\t{} ".format(dso_container.name, command_string))

# 5) Execute command
command_res=dso_container.exec_run(command_string, stdout=True, stderr=True, stdin=False, tty=False, privileged=False, user='', detach=False, stream=False, socket=False, environment=None, workdir=None, demux=False)
if command_res[0]==0:
    print("\t Done!")
else:
    print("\tIt was not possible, error code: {}".format(command_res))


# 6) Remove the datasets from the container
print("\nRemoving {} from {}".format(container_actual_data,dso_container.name))
rm_dataset_cmd = "".join(["bash -c \"","rm -r", " ", str(container_actual_data),"\""])
command_res = dso_container.exec_run(rm_dataset_cmd, stdout=True, stderr=True, stdin=False, tty=False, privileged=False, user='', detach=False, stream=False, socket=False, environment=None, workdir=None, demux=False)
if command_res[0]==0:
    print("\t Done!")
else:
    print("\tIt was not possible, error code: {}".format(command_res))


# 7) Rename the result (a), Compress (b) and copy back to the session folder (c). (d) Finally remove results from container

# (a)
result_filename_remote_ext = "".join([curr_dataset, "_", "DSO.txt"])
cmd_ren= "".join(["bash -c \"", dso_ch_dir_string, " && ", "mv", " ", dso_container_config_algorithm["output_file"], " ", result_filename_remote_ext, "\""])
print("\nRenaming the results, using the command: {}".format(cmd_ren))
command_res = dso_container.exec_run(cmd_ren, stdout=True, stderr=True, stdin=False, tty=False, privileged=False, user='', detach=False, stream=False, socket=False, environment=None, workdir=None, demux=False)
if command_res[0]==0:
    print("\tDone!")
else:
    print("\tIt was not possible, error code: {}".format(command_res))

# (b)
tar_file = "".join([result_filename_remote_ext, ".tar.gz"])
cmd_cmpr_remote = "".join(["bash -c \"", dso_ch_dir_string, " && ", "tar -cvzf", " ", tar_file, " ", result_filename_remote_ext, "\""])
print("\ncompressing the output result command: {}".format(cmd_cmpr_remote))
command_res = dso_container.exec_run(cmd_cmpr_remote, stdout=True, stderr=True, stdin=False, tty=False, privileged=False, user='', detach=False, stream=False, socket=False, environment=None, workdir=None, demux=False)
if command_res[0]==0:
    print("\tDone!")
else:
    print("\tIt was not possible, error code: {}".format(command_res))

# (c)
result_cmpr_local = os.path.join(dataset_out_folder, tar_file)
result_filename_remote_cmpr = os.path.join(dso_bin_path, tar_file)

f = open(result_cmpr_local, 'wb')
print("\nTransferring the file {} ----> {}".format(result_filename_remote_cmpr, result_cmpr_local))
try:
    bits, stat = dso_container.get_archive(result_filename_remote_cmpr)
    for chunk in bits:
        f.write(chunk)
        f.close()
except Exception as error:
    print("\tFatal Error: {}".format(error))
    sys.exit(1)
print("\tDone!\n\tStats:{}".format(stat))

# (d)
result_filename_remote = os.path.join(dso_bin_path, result_filename_remote_ext)
remove_remote_res_cmd = "".join(["bash -c \"", dso_ch_dir_string, " && ", "rm", " ", result_filename_remote, " ", result_filename_remote_cmpr, "\""])
print("\nRemoving the following file from remote container: \t\n- {} \t\n- {}".format(result_filename_remote, result_filename_remote_cmpr))
command_res = dso_container.exec_run(remove_remote_res_cmd, stdout=True, stderr=True, stdin=False, tty=False, privileged=False, user='', detach=False, stream=False, socket=False, environment=None, workdir=None, demux=False)
if command_res[0]==0:
    print("\tDone!")
else:
    print("\tIt was not possible, error code: {}".format(command_res))

# (e) Extract file local and remove the tar file
result_local_final = os.path.join(dataset_out_folder,result_filename_remote_ext)
print("\nDecompressing and removing {}".format(result_cmpr_local))
try:
    with tarfile.open(result_cmpr_local, "r") as tar:
        tar.extractall(result_local_final)
        tar.close()
        os.remove(result_cmpr_local)
        print("\tDone!")
except Exception as error:
    print("\tError while decompressing-> {}".format(error))





**************************************************
 Processing with DSO the Corridor_A_L dataset
**************************************************

Creating Dataset folder: /home/croc/Documenti/Repositories/evoTools/notebooks/session_29_09_2023_15_43_34/Corridor_A_L
	Done!
Copying the gt file /home/croc/Documenti/Trajectories/excasi/Datasets/TUM Format/Corridor_A_L/Corridor_A_L_GT.txt ----> /home/croc/Documenti/Repositories/evoTools/notebooks/session_29_09_2023_15_43_34/Corridor_A_L
	Done!
Copying '/home/croc/Documenti/Trajectories/excasi/Datasets/TUM Format/Corridor_A_L' in "/root/Archive/Dataset/" 
	 - 1) Compressing /home/croc/Documenti/Repositories/evoTools/notebooks/temp/curr_dataset.tar.gz
		Done!
	 - 2) Copying into DSO containter: test_dso
		Done!
	 - 3) Removing the temp file:/home/croc/Documenti/Repositories/evoTools/notebooks/temp/curr_dataset.tar.gz
		Done!

On container "test_dso" execute the following command:  
	bash -c "cd /root/Programs/dso/build/bin/ && ./dso_datase

ValueError: embedded null byte

In [55]:
result_cmpr_local = os.path.join(dataset_out_folder, tar_file)
result_filename_remote_cmpr = os.path.join(dso_bin_path, tar_file)

f = open(result_cmpr_local, 'wb')
print("\nTransferring the file {} ----> {}".format(result_filename_remote_cmpr, result_cmpr_local))
try:
    bits, stat = dso_container.get_archive(result_filename_remote_cmpr)
    for chunk in bits:
        f.write(chunk)
        f.close()
except Exception as error:
    print("\tFatal Error: {}".format(error))
    sys.exit(1)
print("\tDone!\n\tStats:{}".format(stat))


Transferring the file /root/Programs/dso/build/bin/Corridor_A_L_DSO.txt.tar.gz ----> /home/croc/Documenti/Repositories/evoTools/notebooks/session_29_09_2023_15_43_34/Corridor_A_L/Corridor_A_L_DSO.txt.tar.gz
	Done!
	Stats:{'name': 'Corridor_A_L_DSO.txt.tar.gz', 'size': 17626, 'mode': 420, 'mtime': '2023-09-29T15:44:31.492731796+02:00', 'linkTarget': ''}


In [56]:
result_local_final = os.path.join(dataset_out_folder,result_filename_remote_ext)
print("\nDecompressing and removing {}".format(result_cmpr_local))
try:
    with tarfile.open(result_cmpr_local, "r") as tar:
        tar.extractall(result_local_final)
        tar.close()
        os.remove(result_cmpr_local)
        print("\tDone!")
except Exception as error:
    print("\tError while decompressing-> {}".format(error))


Decompressing and removing /home/croc/Documenti/Repositories/evoTools/notebooks/session_29_09_2023_15_43_34/Corridor_A_L/Corridor_A_L_DSO.txt.tar.gz
	Done!
